In [72]:
import numpy as np
import pandas as pd

books = pd.read_csv('Books.csv')
users = pd.read_csv('Users.csv')
ratings = pd.read_csv('Ratings.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [73]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [74]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [75]:
print(books.shape)
print(ratings.shape)
print(users.shape)

(271360, 8)
(1149780, 3)
(278858, 3)


In [76]:
books.isnull().sum()

ISBN                   0
Book-Title             0
Book-Author            1
Year-Of-Publication    0
Publisher              2
Image-URL-S            0
Image-URL-M            0
Image-URL-L            3
dtype: int64

In [77]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

In [78]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [79]:
books.duplicated().sum()

0

In [80]:
ratings.duplicated().sum()

0

In [81]:
users.duplicated().sum()

0

## Popularity Based Recommender System

In [82]:
ratings_with_name = ratings.merge(books,on='ISBN')

num_rating_df = ratings_with_name.groupby('Book-Title').count()['Book-Rating'].reset_index()
num_rating_df.rename(columns={'Book-Rating':'num_ratings'},inplace=True)
num_rating_df

,Book-Title,num_ratings
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241066,Ã?Â?lpiraten.,2
241067,Ã?Â?rger mit Produkt X. Roman.,4
241068,Ã?Â?sterlich leben.,1
241069,Ã?Â?stlich der Berge.,3


In [83]:
avg_rating_df = ratings_with_name.groupby('Book-Title').mean()['Book-Rating'].reset_index()
avg_rating_df.rename(columns={'Book-Rating':'avg_rating'},inplace=True)
avg_rating_df

,Book-Title,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,2.250000
1,Always Have Popsicles,0.000000
2,Apple Magic (The Collector's series),0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,0.000000
...,...,...
241066,Ã?Â?lpiraten.,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,5.250000
241068,Ã?Â?sterlich leben.,7.000000
241069,Ã?Â?stlich der Berge.,2.666667


In [84]:
popular_df = num_rating_df.merge(avg_rating_df,on='Book-Title')
popular_df

,Book-Title,num_ratings,avg_rating
0,A Light in the Storm: The Civil War Diary of ...,4,2.250000
1,Always Have Popsicles,1,0.000000
2,Apple Magic (The Collector's series),1,0.000000
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1,8.000000
4,Beyond IBM: Leadership Marketing and Finance ...,1,0.000000
...,...,...,...
241066,Ã?Â?lpiraten.,2,0.000000
241067,Ã?Â?rger mit Produkt X. Roman.,4,5.250000
241068,Ã?Â?sterlich leben.,1,7.000000
241069,Ã?Â?stlich der Berge.,3,2.666667


In [85]:
popular_df = popular_df[popular_df['num_ratings']>=250].sort_values('avg_rating',ascending=False).head(50)

In [86]:
popular_df = popular_df.merge(books,on='Book-Title').drop_duplicates('Book-Title')[['Book-Title','Book-Author','Image-URL-M','num_ratings','avg_rating']]

In [87]:
popular_df['Image-URL-M'][0]

'http://images.amazon.com/images/P/0439136350.01.MZZZZZZZ.jpg'

## Collaborative Filtering Based Recommender System

In [88]:
x = ratings_with_name.groupby('User-ID').count()['Book-Rating'] > 200
padhe_likhe_users = x[x].index

In [89]:
filtered_rating = ratings_with_name[ratings_with_name['User-ID'].isin(padhe_likhe_users)]

In [90]:
y = filtered_rating.groupby('Book-Title').count()['Book-Rating']>=50
famous_books = y[y].index

In [91]:
final_ratings = filtered_rating[filtered_rating['Book-Title'].isin(famous_books)]

In [92]:
pt = final_ratings.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [93]:
pt.fillna(0,inplace=True)

In [94]:
pt

User-ID,254,2276,2766,2977,3363,4017,4385,6251,6323,6543,...,271705,273979,274004,274061,274301,274308,275970,277427,277639,278418
Book-Title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Bend in the Road,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
from sklearn.metrics.pairwise import cosine_similarity

In [96]:
similarity_scores = cosine_similarity(pt)

In [97]:
similarity_scores.shape

(706, 706)

In [98]:
def recommend(book_name):
    # index fetch
    index = np.where(pt.index==book_name)[0][0]
    similar_items = sorted(list(enumerate(similarity_scores[index])),key=lambda x:x[1],reverse=True)[1:5]
    
    data = []
    for i in similar_items:
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Title'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Book-Author'].values))
        item.extend(list(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values))
        
        data.append(item)
    
    return data

In [99]:
recommend('1984')

[['Animal Farm',
  'George Orwell',
  'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'],
 ["The Handmaid's Tale",
  'Margaret Atwood',
  'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'],
 ['Brave New World',
  'Aldous Huxley',
  'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'],
 ['The Vampire Lestat (Vampire Chronicles, Book II)',
  'ANNE RICE',
  'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg']]

In [100]:
pt.index[545]

"The Handmaid's Tale"

In [101]:
import pickle
pickle.dump(popular_df,open('popular.pkl','wb'))

In [102]:
books.drop_duplicates('Book-Title')

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271354,0449906736,Flashpoints: Promise and Peril in a New World,Robin Wright,1993,Ballantine Books,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...,http://images.amazon.com/images/P/0449906736.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [103]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(similarity_scores,open('similarity_scores.pkl','wb'))

In [4]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

# Preprocessing
books_with_ratings = pd.merge(books, ratings, on='ISBN')
books_with_ratings = books_with_ratings.dropna(subset=['Book-Title'])

# Content-Based Recommender: Simplified
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])
indices = pd.Series(books_with_ratings.index, index=books_with_ratings['Book-Title']).drop_duplicates()

# Collaborative Filtering
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    idx = indices[book_title]
    
    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(count_matrix[idx], count_matrix)
    
    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:50]  # Increase to get a larger pool of similar books
    book_indices = [i[0] for i in sim_scores]

    # Predict ratings for these books for the given user
    books_predictions = [(books_with_ratings['Book-Title'].iloc[i], svd.predict(user_id, books_with_ratings['ISBN'].iloc[i]).est) for i in book_indices]
    books_predictions.sort(key=lambda x: x[1], reverse=True)  # Sort by predicted rating

    # Select top 10
    top_books = [book[0] for book in books_predictions[:10]]
    
    return top_books

# Example usage
print(get_recommendations(276726, "Classical Mythology"))



C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Classical Mythology', 'Mythology', 'Mythology', 'Mythology', 'Mythology', 'Mythology', 'Mythology', 'Mythology', 'Mythology', 'Mythology']


In [5]:
# Model Evaluation
def evaluate_model(model, data):
    cross_val_results = cross_validate(model, data, measures=['RMSE'], cv=5, verbose=True)
    return cross_val_results

evaluation_results = evaluate_model(svd, data)
print(evaluation_results)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.5044  3.5024  3.4987  3.5113  3.5028  3.5039  0.0042  
Fit time          19.52   20.28   21.09   20.34   20.62   20.37   0.51    
Test time         3.44    2.63    2.48    2.48    2.63    2.73    0.36    
{'test_rmse': array([3.50435508, 3.50241154, 3.49868794, 3.51134093, 3.50277328]), 'fit_time': (19.516736030578613, 20.278340816497803, 21.091652631759644, 20.337438583374023, 20.61928391456604), 'test_time': (3.439271926879883, 2.63273024559021, 2.4783668518066406, 2.482888698577881, 2.627638816833496)}


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

# Preprocessing
books_with_ratings = pd.merge(books, ratings, on='ISBN')
books_with_ratings = books_with_ratings.dropna(subset=['Book-Title'])

# Content-Based Recommender: Simplified
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])
indices = pd.Series(books_with_ratings.index, index=books_with_ratings['Book-Title']).drop_duplicates()

# Collaborative Filtering
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(count_matrix[idx], count_matrix)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]  # Increased to get a larger pool of similar books

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings['Book-Title'].iloc[i]
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings['ISBN'].iloc[i]).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
print(get_recommendations(276726, "Classical Mythology"))



C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Mythology (Cliffs Notes)', 'Age of Fable or Beauties of Mythology (Bulfinch Mythology)', "Bulfinch's Mythology", 'Classical mythology', 'Brush Up Your Mythology!', 'Whos Who In Mythology', "Crowell's Handbook of Classical Mythology (A Crowell reference book)", 'Greek Mythology', 'Illustrated Dictionary of Mythology', 'The Dictionary of Classical Mythology']


In [13]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD, accuracy
from surprise.model_selection import train_test_split, cross_validate

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

# Filter out zero ratings
filtered_ratings = ratings[ratings['Book-Rating'] > 0]

# Merge books with filtered ratings
books_with_ratings = pd.merge(books, filtered_ratings, on='ISBN')


# Preprocessing
books_with_ratings = books_with_ratings.dropna(subset=['Book-Title'])

# Content-Based Recommender: Simplified
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])
indices = pd.Series(books_with_ratings.index, index=books_with_ratings['Book-Title']).drop_duplicates()

# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    if book_title not in indices:
        return f"Book title '{book_title}' not found in the dataset."

    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(count_matrix[idx], count_matrix)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]  # Increased pool of similar books

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings['Book-Title'].iloc[i]
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings['ISBN'].iloc[i]).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
recommended_books = get_recommendations(276726, "Classical Mythology")
print(recommended_books)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Mythology', "Bulfinch's Mythology (Laurel Classic)", 'The Song of Eve: Mythology and Symbols of the Goddess', 'The Well-Educated Mind: A Guide to the Classical Education You Never Had', 'The Classical Greek Reader', 'Dictionary of Mythology: Mainly Classical', 'The Illustrated Guide to Celtic Mythology', 'The Oxford History of the Classical World', 'Classical Music for Dummies', 'The Oxford Companion to Classical Civilization']


In [21]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

# Filter out zero ratings
filtered_ratings = ratings[ratings['Book-Rating'] > 0]

# Merge books with filtered ratings
books_with_ratings = pd.merge(books, filtered_ratings, on='ISBN')

# Define minimum number of ratings required for books and users
min_book_ratings = 1
min_user_ratings = 1

# Filter books and users based on defined thresholds
book_counts = books_with_ratings['ISBN'].value_counts()
user_counts = books_with_ratings['User-ID'].value_counts()
books_with_ratings = books_with_ratings[books_with_ratings['ISBN'].isin(book_counts[book_counts >= min_book_ratings].index)]
books_with_ratings = books_with_ratings[books_with_ratings['User-ID'].isin(user_counts[user_counts >= min_user_ratings].index)]

# Preprocessing to drop NaNs in 'Book-Title'
books_with_ratings = books_with_ratings.dropna(subset=['Book-Title'])

# Content-Based Recommender: Simplified
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])
indices = pd.Series(books_with_ratings.index, index=books_with_ratings['Book-Title']).drop_duplicates()

# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    if book_title not in indices:
        return f"Book title '{book_title}' not found in the dataset."

    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(count_matrix[idx], count_matrix)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]  # Increased pool of similar books

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings['Book-Title'].iloc[i]
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings['ISBN'].iloc[i]).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
recommended_books = get_recommendations(276726, "Classical Mythology")
print(recommended_books)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Mythology', "Bulfinch's Mythology (Laurel Classic)", 'The Well-Educated Mind: A Guide to the Classical Education You Never Had', 'Dictionary of Mythology: Mainly Classical', 'Mythology (Collins Gem)', 'Mythology: Timeless Tales of Gods and Heroes', 'Classical mythology', "Bulfinch's Mythology", 'Hawaiian Mythology', "Who's Who in Non-Classical Mythology"]


The code provided is an example of a Hybrid Recommendation System. It combines elements of both Content-Based Filtering and Collaborative Filtering to generate recommendations. Let's break down how each part contributes to the hybrid model:

Content-Based Filtering: This part is achieved through the use of CountVectorizer to transform the book titles into a count matrix and then applying cosine similarity to find books with similar titles. This method focuses on the content (in this case, the titles of the books) to find items similar to the user's past preferences or specific input (the given book title).

Collaborative Filtering: This aspect is implemented using the surprise library, where the SVD (Singular Value Decomposition) algorithm is used to predict how a user might rate books based on how similar users have rated them. This method leverages the collective ratings of users to make recommendations.

Hybrid Approach: The system first finds books similar to the input book title (content-based) and then filters and ranks these books based on the predicted preferences of the user (collaborative filtering). The final recommendation list is sorted based on these predicted ratings.

By combining these two approaches, the hybrid model aims to leverage both the content features of the items (books in this case) and the user-item interaction data, providing a more nuanced set of recommendations than either method would on its own. This approach can help alleviate some common issues in recommendation systems, such as the cold start problem and the limitation of recommending items only based on user interaction history or item features.

In [18]:
import pandas as pd
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')

# Filter out zero ratings
filtered_ratings = ratings[ratings['Book-Rating'] > 0]

# Merge books with filtered ratings
books_with_ratings = pd.merge(books, filtered_ratings, on='ISBN')

# Define minimum number of ratings required for books and users
min_book_ratings = 1
min_user_ratings = 1

# Filter books and users based on defined thresholds
book_counts = books_with_ratings['ISBN'].value_counts()
user_counts = books_with_ratings['User-ID'].value_counts()
books_with_ratings = books_with_ratings[books_with_ratings['ISBN'].isin(book_counts[book_counts >= min_book_ratings].index)]
books_with_ratings = books_with_ratings[books_with_ratings['User-ID'].isin(user_counts[user_counts >= min_user_ratings].index)]

# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)
svd = SVD()

# Evaluate the model
cross_val_results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)

# Output the results
print(cross_val_results)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6308  1.6417  1.6374  1.6335  1.6371  1.6361  0.0037  
Fit time          7.07    7.24    7.73    7.52    7.31    7.38    0.23    
Test time         0.91    0.91    0.60    0.94    0.89    0.85    0.13    
{'test_rmse': array([1.63082637, 1.64173699, 1.6374449 , 1.63350614, 1.63711106]), 'fit_time': (7.068379878997803, 7.240798711776733, 7.732605934143066, 7.5235371589660645, 7.314286231994629), 'test_time': (0.9125139713287354, 0.9053792953491211, 0.6012551784515381, 0.9425902366638184, 0.8931059837341309)}



The evaluation results you provided indicate an RMSE (Root Mean Square Error) with a mean of approximately 1.6361 across the 5 folds of cross-validation. Here's how to interpret these results:

RMSE (Root Mean Square Error): RMSE is a standard metric for evaluating the accuracy of a prediction model. It measures the average magnitude of the prediction errors, i.e., the differences between the predicted values and the actual values. An RMSE of 0 would mean perfect predictions, which is typically not achievable in real-world scenarios.

Mean RMSE of 1.6361: This value suggests that, on average, the predicted ratings by your model differ from the actual ratings by about 1.6361 points on the rating scale (assuming your ratings are on a scale from 1 to 10).

Standard Deviation of RMSE (0.0037): The low standard deviation indicates that the model's performance is relatively consistent across different subsets of your data.

Is the Model Good?
Context of RMSE Value: The acceptability of the RMSE value depends on the context and the specific application. In a 10-point rating system, an average error of around 1.6 might be considered reasonable, but this can vary depending on the complexity of the data and the domain. For instance, in some scenarios, even a small error can be significant, while in others, a larger error is acceptable.

Comparative Analysis: To better understand if this is a good RMSE value, you could compare it with baseline models or other algorithms. If your model performs significantly better than a basic model (like one that always predicts the average rating), it indicates a good model.

User Satisfaction: Ultimately, the success of a recommendation system is also measured by user satisfaction, which can be gauged through user feedback, engagement metrics, and practical usability tests.

Room for Improvement: If you aim to improve the model further, consider experimenting with different algorithms, tuning hyperparameters, or enriching the dataset with more features.

In summary, while the RMSE value provides a quantitative measure of your model's accuracy, assessing its quality also requires qualitative considerations and possibly comparisons with other models or baselines.

# NEW TRAILS BY ADDING MORE INPUTS

In [26]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

users['Location'] = users['Location'].str.lower() ## converitng location into lower coses. 
users.loc[:, 'Location'] = users['Location'].str.replace(' ', '', regex=True)
# Merge datasets
books_with_ratings = pd.merge(books, ratings, on='ISBN')
books_with_ratings = pd.merge(books_with_ratings, users, on='User-ID')

books_with_ratings['Year-Of-Publication'] = pd.to_numeric(books_with_ratings['Year-Of-Publication'], errors='coerce')

# Fill missing values
books_with_ratings['Book-Author'].fillna('Unknown', inplace=True)
books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)
books_with_ratings['Location'].fillna('Unknown', inplace=True)


# Convert 'Year-Of-Publication' to numeric, handling non-numeric entries

books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)

# Preprocessing pipeline for additional features
preprocessor = ColumnTransformer(
    transformers=[
        ('author', OneHotEncoder(handle_unknown='ignore'), ['Book-Author']),
        ('location', OneHotEncoder(handle_unknown='ignore'), ['Location']),
        ('year', StandardScaler(), ['Year-Of-Publication'])
    ])

# Fit and transform the data
additional_features = preprocessor.fit_transform(books_with_ratings)

# Content-Based Recommender for Book Titles
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])

# Combine title features with additional features
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
combined_features = hstack([count_matrix, additional_features])

combined_features_csr = csr_matrix(combined_features)


# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'ISBN', 'Book-Rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Update indices after combining features
indices = pd.Series(range(len(books_with_ratings)), index=books_with_ratings['Book-Title']).drop_duplicates()

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    if book_title not in indices:
        return f"Book title '{book_title}' not found in the dataset."

    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(combined_features_csr[idx], combined_features_csr)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings.iloc[i]['Book-Title']
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings.iloc[i]['ISBN']).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
print(get_recommendations(276726, "Classical Mythology"))


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Into the Wild', 'Slaughterhouse-Five', 'The Client', 'Among Schoolchildren', 'Three Fates', 'Slackjaw', 'The Summons', 'The Moviegoer', 'The Bottoms', 'Shopgirl']


In [ ]:
Two codes for evaluation, one for collabrative filtering another for content based 

In [27]:

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

svd = SVD()

# Evaluate the model using cross-validation
cross_val_results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)
print(cross_val_results)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    3.5022  3.5004  3.4960  3.5081  3.4942  3.5002  0.0049  
Fit time          19.32   24.44   21.62   22.84   19.97   21.64   1.87    
Test time         4.79    3.61    3.38    3.19    2.78    3.55    0.68    
{'test_rmse': array([3.50217204, 3.50037164, 3.49603394, 3.50809783, 3.49422042]), 'fit_time': (19.31854009628296, 24.43724274635315, 21.617775678634644, 22.8384530544281, 19.973925590515137), 'test_time': (4.788440704345703, 3.611894130706787, 3.383833646774292, 3.190612316131592, 2.77546763420105)}


In [30]:
import numpy as np

def get_feature_vector(book_title, feature_matrix, book_indices):
    # Retrieve the row from the feature matrix corresponding to the given book title
    idx = book_indices[book_title]
    return feature_matrix[idx]

def calculate_average_similarity(input_book_vector, recommended_books, feature_matrix, book_indices):
    similarities = []
    for book in recommended_books:
        book_vector = get_feature_vector(book, feature_matrix, book_indices)
        if book_vector is not None:
            similarity = cosine_similarity(input_book_vector, book_vector)
            similarities.append(similarity[0][0])
    return np.mean(similarities) if similarities else 0

# Example usage
input_book_vector = get_feature_vector("Classical Mythology", combined_features_csr, indices)
recommended_books = get_recommendations(276726, "Classical Mythology")
average_similarity = calculate_average_similarity(input_book_vector, recommended_books, combined_features_csr, indices)


In [31]:
average_similarity

0.006012063945992824

In [32]:
def categorize_rating(rating):
    if rating <= 4:
        return 'Low'
    elif rating <= 7:
        return 'Medium'
    else:
        return 'High'



In [39]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

users['Location'] = users['Location'].str.lower() ## converitng location into lower coses. 
users.loc[:, 'Location'] = users['Location'].str.replace(' ', '', regex=True)

# Split and clean the 'Location' column in users_df
users[['City', 'State', 'Country']] = users['Location'].str.split(',', n=2, expand=True)
users['City'] = users['City'].str.strip()
users['State'] = users['State'].str.strip()
users['Country'] = users['Country'].str.strip()

# Merge datasets
books_with_ratings = pd.merge(books, ratings, on='ISBN')
books_with_ratings = pd.merge(books_with_ratings, users, on='User-ID')

books_with_ratings['Rating-Category'] = books_with_ratings['Book-Rating'].apply(categorize_rating)
rating_to_numeric = {'Low': 1, 'Medium': 2, 'High': 3}
books_with_ratings['Numeric-Rating-Category'] = books_with_ratings['Rating-Category'].map(rating_to_numeric)

median_age = books_with_ratings['Age'].median()
books_with_ratings['Age'].fillna(median_age, inplace=True)


books_with_ratings['Year-Of-Publication'] = pd.to_numeric(books_with_ratings['Year-Of-Publication'], errors='coerce')

# Fill missing values
books_with_ratings['Book-Author'].fillna('Unknown', inplace=True)
books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)
books_with_ratings['Location'].fillna('Unknown', inplace=True)


# Convert 'Year-Of-Publication' to numeric, handling non-numeric entries

books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)

# Preprocessing pipeline for additional features
preprocessor = ColumnTransformer(
    transformers=[
        ('author', OneHotEncoder(handle_unknown='ignore'), ['Book-Author']),
        ('location', OneHotEncoder(handle_unknown='ignore'), ['Location']),
        ('year', StandardScaler(), ['Year-Of-Publication'])
    ])

# Fit and transform the data
additional_features = preprocessor.fit_transform(books_with_ratings)

# Content-Based Recommender for Book Titles
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])

# Combine title features with additional features
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
combined_features = hstack([count_matrix, additional_features])

combined_features_csr = csr_matrix(combined_features)


# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'ISBN', 'Numeric-Rating-Category']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Update indices after combining features
indices = pd.Series(range(len(books_with_ratings)), index=books_with_ratings['Book-Title']).drop_duplicates()

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    if book_title not in indices:
        return f"Book title '{book_title}' not found in the dataset."

    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(combined_features_csr[idx], combined_features_csr)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings.iloc[i]['Book-Title']
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings.iloc[i]['ISBN']).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
print(get_recommendations(276726, "Classical Mythology"))


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Slaughterhouse-Five', 'The Partner', 'The Associate', 'Three Fates', "Who's Who in Non-Classical Mythology", 'Unexpected', 'Shopgirl', 'Dreamcatcher', 'The Client', 'The Summons']


In [40]:

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

svd = SVD()

# Evaluate the model using cross-validation
cross_val_results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)
print(cross_val_results)



import numpy as np

def get_feature_vector(book_title, feature_matrix, book_indices):
    # Retrieve the row from the feature matrix corresponding to the given book title
    idx = book_indices[book_title]
    return feature_matrix[idx]

def calculate_average_similarity(input_book_vector, recommended_books, feature_matrix, book_indices):
    similarities = []
    for book in recommended_books:
        book_vector = get_feature_vector(book, feature_matrix, book_indices)
        if book_vector is not None:
            similarity = cosine_similarity(input_book_vector, book_vector)
            similarities.append(similarity[0][0])
    return np.mean(similarities) if similarities else 0

# Example usage
input_book_vector = get_feature_vector("Classical Mythology", combined_features_csr, indices)
recommended_books = get_recommendations(276726, "Classical Mythology")
average_similarity = calculate_average_similarity(input_book_vector, recommended_books, combined_features_csr, indices)
average_similarity

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7355  0.7385  0.7385  0.7372  0.7370  0.7373  0.0011  
Fit time          14.26   14.49   14.67   15.43   15.04   14.78   0.41    
Test time         1.45    2.63    1.42    3.22    2.79    2.30    0.73    
{'test_rmse': array([0.735502  , 0.73845988, 0.7384513 , 0.73718041, 0.73695016]), 'fit_time': (14.264035701751709, 14.488046646118164, 14.670043230056763, 15.433007717132568, 15.041014194488525), 'test_time': (1.4480071067810059, 2.6349918842315674, 1.4150056838989258, 3.216000556945801, 2.7899835109710693)}


0.005560293901847928

In [45]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

users['Location'] = users['Location'].str.lower() ## converitng location into lower coses. 
users.loc[:, 'Location'] = users['Location'].str.replace(' ', '', regex=True)

# Split and clean the 'Location' column in users_df
users[['City', 'State', 'Country']] = users['Location'].str.split(',', n=2, expand=True)
users['City'] = users['City'].str.strip()
users['State'] = users['State'].str.strip()
users['Country'] = users['Country'].str.strip()

# Merge datasets
books_with_ratings = pd.merge(books, ratings, on='ISBN')
books_with_ratings = pd.merge(books_with_ratings, users, on='User-ID')

books_with_ratings['Rating-Category'] = books_with_ratings['Book-Rating'].apply(categorize_rating)
rating_to_numeric = {'Low': 1, 'Medium': 2, 'High': 3}
books_with_ratings['Numeric-Rating-Category'] = books_with_ratings['Rating-Category'].map(rating_to_numeric)

median_age = books_with_ratings['Age'].median()
books_with_ratings['Age'].fillna(median_age, inplace=True)


books_with_ratings['Year-Of-Publication'] = pd.to_numeric(books_with_ratings['Year-Of-Publication'], errors='coerce')

# Fill missing values
books_with_ratings['Book-Author'].fillna('Unknown', inplace=True)
books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)
books_with_ratings['Location'].fillna('Unknown', inplace=True)


# Convert 'Year-Of-Publication' to numeric, handling non-numeric entries

books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)

# Preprocessing pipeline for additional features
preprocessor = ColumnTransformer(
    transformers=[
        ('author', OneHotEncoder(handle_unknown='ignore'), ['Book-Author']),
        ('location', OneHotEncoder(handle_unknown='ignore'), ['State']),
        ('year', StandardScaler(), ['Year-Of-Publication'])
    ])

# Fit and transform the data
additional_features = preprocessor.fit_transform(books_with_ratings)

# Content-Based Recommender for Book Titles
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])

# Combine title features with additional features
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
combined_features = hstack([count_matrix, additional_features])

combined_features_csr = csr_matrix(combined_features)


# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'Book-Title', 'Numeric-Rating-Category']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Update indices after combining features
indices = pd.Series(range(len(books_with_ratings)), index=books_with_ratings['Book-Title']).drop_duplicates()

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    if book_title not in indices:
        return f"Book title '{book_title}' not found in the dataset."

    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(combined_features_csr[idx], combined_features_csr)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings.iloc[i]['Book-Title']
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings.iloc[i]['ISBN']).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
print(get_recommendations(276726, "Classical Mythology"))


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Classical Mythology', 'Mythology', 'The Penguin Dictionary of Classical Mythology (Reference Books)', "Bulfinch's Mythology", 'Classical Music', 'Classical mythology', 'Comparative Mythology', 'Hawaiian Mythology', 'Greek mythology', "Mythology and You : Classical Mythology and its Relevance in Today's World"]


In [46]:

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

svd = SVD()

# Evaluate the model using cross-validation
cross_val_results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)
print(cross_val_results)



import numpy as np

def get_feature_vector(book_title, feature_matrix, book_indices):
    # Retrieve the row from the feature matrix corresponding to the given book title
    idx = book_indices[book_title]
    return feature_matrix[idx]

def calculate_average_similarity(input_book_vector, recommended_books, feature_matrix, book_indices):
    similarities = []
    for book in recommended_books:
        book_vector = get_feature_vector(book, feature_matrix, book_indices)
        if book_vector is not None:
            similarity = cosine_similarity(input_book_vector, book_vector)
            similarities.append(similarity[0][0])
    return np.mean(similarities) if similarities else 0

# Example usage
input_book_vector = get_feature_vector("Classical Mythology", combined_features_csr, indices)
recommended_books = get_recommendations(276726, "Classical Mythology")
average_similarity = calculate_average_similarity(input_book_vector, recommended_books, combined_features_csr, indices)


Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7359  0.7380  0.7370  0.7372  0.7359  0.7368  0.0008  
Fit time          22.79   25.89   25.40   24.83   25.90   24.96   1.15    
Test time         4.22    4.42    3.16    3.47    3.96    3.85    0.47    
{'test_rmse': array([0.73593546, 0.73799244, 0.73696184, 0.73721835, 0.73594706]), 'fit_time': (22.79499578475952, 25.89400291442871, 25.396962642669678, 24.82599973678589, 25.900165796279907), 'test_time': (4.22303581237793, 4.419001579284668, 3.1600005626678467, 3.473998785018921, 3.964035987854004)}


In [47]:
average_similarity

0.45576695729308214

In [67]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

users['Location'] = users['Location'].str.lower() ## converitng location into lower coses. 
users.loc[:, 'Location'] = users['Location'].str.replace(' ', '', regex=True)

# Split and clean the 'Location' column in users_df
users[['City', 'State', 'Country']] = users['Location'].str.split(',', n=2, expand=True)
users['City'] = users['City'].str.strip()
users['State'] = users['State'].str.strip()
users['Country'] = users['Country'].str.strip()

# Merge datasets
books_with_ratings = pd.merge(books, ratings, on='ISBN')
books_with_ratings = pd.merge(books_with_ratings, users, on='User-ID')

books_with_ratings['Rating-Category'] = books_with_ratings['Book-Rating'].apply(categorize_rating)
rating_to_numeric = {'Low': 1, 'Medium': 2, 'High': 3}
books_with_ratings['Numeric-Rating-Category'] = books_with_ratings['Rating-Category'].map(rating_to_numeric)

median_age = books_with_ratings['Age'].median()
books_with_ratings['Age'].fillna(median_age, inplace=True)


books_with_ratings['Year-Of-Publication'] = pd.to_numeric(books_with_ratings['Year-Of-Publication'], errors='coerce')

# Fill missing values
books_with_ratings['Book-Author'].fillna('Unknown', inplace=True)
books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)
books_with_ratings['Location'].fillna('Unknown', inplace=True)


# Convert 'Year-Of-Publication' to numeric, handling non-numeric entries

books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)

# Preprocessing pipeline for additional features
preprocessor = ColumnTransformer(
    transformers=[
        ('author', OneHotEncoder(handle_unknown='ignore'), ['Book-Author']),
        ('location', OneHotEncoder(handle_unknown='ignore'), ['Location']),
        ('year', StandardScaler(), ['Year-Of-Publication'])
    ])

# Fit and transform the data
additional_features = preprocessor.fit_transform(books_with_ratings)

# Content-Based Recommender for Book Titles
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])

# Combine title features with additional features
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
combined_features = hstack([count_matrix, additional_features])

combined_features_csr = csr_matrix(combined_features)


# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'Book-Title', 'Numeric-Rating-Category']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Update indices after combining features
indices = pd.Series(range(len(books_with_ratings)), index=books_with_ratings['Book-Title']).drop_duplicates()

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    if book_title not in indices:
        return f"Book title '{book_title}' not found in the dataset."

    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(combined_features_csr[idx], combined_features_csr)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings.iloc[i]['Book-Title']
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings.iloc[i]['ISBN']).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
print(get_recommendations(276726, "Rebecca"))


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Rebecca', 'The Key to Rebecca', 'Rebecca (Compact Books)', "Rebecca (The world's best reading)", 'My Cousin Rachel (Common Reader Editions)', 'Q', 'Cry No More', 'Under Fire', 'Now You See It . . .', 'Once']


In [68]:

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

svd = SVD()

# Evaluate the model using cross-validation
cross_val_results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)
print(cross_val_results)



import numpy as np

def get_feature_vector(book_title, feature_matrix, book_indices):
    # Retrieve the row from the feature matrix corresponding to the given book title
    idx = book_indices[book_title]
    return feature_matrix[idx]

def calculate_average_similarity(input_book_vector, recommended_books, feature_matrix, book_indices):
    similarities = []
    for book in recommended_books:
        book_vector = get_feature_vector(book, feature_matrix, book_indices)
        if book_vector is not None:
            similarity = cosine_similarity(input_book_vector, book_vector)
            similarities.append(similarity[0][0])
    return np.mean(similarities) if similarities else 0

# Example usage
input_book_vector = get_feature_vector("Classical Mythology", combined_features_csr, indices)
recommended_books = get_recommendations(276726, "Classical Mythology")
average_similarity = calculate_average_similarity(input_book_vector, recommended_books, combined_features_csr, indices)
average_similarity

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7370  0.7379  0.7353  0.7368  0.7386  0.7371  0.0011  
Fit time          34.65   38.75   37.21   37.33   36.71   36.93   1.32    
Test time         6.52    5.54    6.16    4.65    6.53    5.88    0.71    
{'test_rmse': array([0.73698688, 0.73787366, 0.73530408, 0.73684907, 0.73857673]), 'fit_time': (34.65027379989624, 38.745248317718506, 37.21324896812439, 37.32742714881897, 36.711984395980835), 'test_time': (6.519286632537842, 5.540955066680908, 6.160151958465576, 4.6478376388549805, 6.528259038925171)}


0.41550206306433424

In [70]:
## HYer Arameter Tuning 

from surprise.model_selection import GridSearchCV

param_grid = {
    'n_epochs': [5, 10, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# Use the best parameters for SVD
algo = SVD(n_epochs=gs.best_params['rmse']['n_epochs'], lr_all=gs.best_params['rmse']['lr_all'], reg_all=gs.best_params['rmse']['reg_all'])


0.7392259219930469
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4}


The results from your hyperparameter tuning indicate that the best parameters for the SVD algorithm are:

n_epochs: 20
lr_all: 0.005
reg_all: 0.4
And the lowest RMSE achieved with these parameters is approximately 0.7392.

In [71]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

# Load the dataset (assuming the 'data' variable holds your Dataset object)
reader = Reader(rating_scale=(1, 10))
# Assuming 'books_with_ratings' holds your data
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'Book-Title', 'Numeric-Rating-Category']], reader)

# Initialize the SVD algorithm with the optimized parameters
algo = SVD(n_epochs=20, lr_all=0.005, reg_all=0.4)

# Perform cross-validation and print results
cross_val_results = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7385  0.7367  0.7378  0.7374  0.7370  0.7375  0.0006  
Fit time          15.61   15.55   15.32   15.01   15.53   15.40   0.22    
Test time         44.79   32.95   17.14   14.35   9.50    23.75   13.13   


### Added Unknown to ratings to check the outut

In [48]:
def categorize_rating(rating):
    if rating <= 4:
        return 'Low'
    elif rating <= 7:
        return 'Medium'
    elif rating == 0:
        return 'Unknown'
    else:
        return 'High'



import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

users['Location'] = users['Location'].str.lower() ## converitng location into lower coses. 
users.loc[:, 'Location'] = users['Location'].str.replace(' ', '', regex=True)

# Split and clean the 'Location' column in users_df
users[['City', 'State', 'Country']] = users['Location'].str.split(',', n=2, expand=True)
users['City'] = users['City'].str.strip()
users['State'] = users['State'].str.strip()
users['Country'] = users['Country'].str.strip()

# Merge datasets
books_with_ratings = pd.merge(books, ratings, on='ISBN')
books_with_ratings = pd.merge(books_with_ratings, users, on='User-ID')

books_with_ratings['Rating-Category'] = books_with_ratings['Book-Rating'].apply(categorize_rating)
rating_to_numeric = {'Low': 1, 'Medium': 2, 'High': 3, 'Unknown': 0}
books_with_ratings['Numeric-Rating-Category'] = books_with_ratings['Rating-Category'].map(rating_to_numeric)

median_age = books_with_ratings['Age'].median()
books_with_ratings['Age'].fillna(median_age, inplace=True)


books_with_ratings['Year-Of-Publication'] = pd.to_numeric(books_with_ratings['Year-Of-Publication'], errors='coerce')

# Fill missing values
books_with_ratings['Book-Author'].fillna('Unknown', inplace=True)
books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)
books_with_ratings['Location'].fillna('Unknown', inplace=True)


# Convert 'Year-Of-Publication' to numeric, handling non-numeric entries

books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)

# Preprocessing pipeline for additional features
preprocessor = ColumnTransformer(
    transformers=[
        ('author', OneHotEncoder(handle_unknown='ignore'), ['Book-Author']),
        ('location', OneHotEncoder(handle_unknown='ignore'), ['Location']),
        ('year', StandardScaler(), ['Year-Of-Publication'])
    ])

# Fit and transform the data
additional_features = preprocessor.fit_transform(books_with_ratings)

# Content-Based Recommender for Book Titles
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])

# Combine title features with additional features
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
combined_features = hstack([count_matrix, additional_features])

combined_features_csr = csr_matrix(combined_features)


# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'Book-Title', 'Numeric-Rating-Category']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Update indices after combining features
indices = pd.Series(range(len(books_with_ratings)), index=books_with_ratings['Book-Title']).drop_duplicates()

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    if book_title not in indices:
        return f"Book title '{book_title}' not found in the dataset."

    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(combined_features_csr[idx], combined_features_csr)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings.iloc[i]['Book-Title']
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings.iloc[i]['ISBN']).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
print(get_recommendations(276726, "Rebecca"))


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Rebecca', 'The Key to Rebecca', 'Rebecca (Compact Books)', "Rebecca (The world's best reading)", 'My Cousin Rachel (Common Reader Editions)', 'Q', 'Cry No More', 'Under Fire', 'Now You See It . . .', 'Once']


In [49]:

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

svd = SVD()

# Evaluate the model using cross-validation
cross_val_results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)
print(cross_val_results)



import numpy as np

def get_feature_vector(book_title, feature_matrix, book_indices):
    # Retrieve the row from the feature matrix corresponding to the given book title
    idx = book_indices[book_title]
    return feature_matrix[idx]

def calculate_average_similarity(input_book_vector, recommended_books, feature_matrix, book_indices):
    similarities = []
    for book in recommended_books:
        book_vector = get_feature_vector(book, feature_matrix, book_indices)
        if book_vector is not None:
            similarity = cosine_similarity(input_book_vector, book_vector)
            similarities.append(similarity[0][0])
    return np.mean(similarities) if similarities else 0

# Example usage
input_book_vector = get_feature_vector("Classical Mythology", combined_features_csr, indices)
recommended_books = get_recommendations(276726, "Classical Mythology")
average_similarity = calculate_average_similarity(input_book_vector, recommended_books, combined_features_csr, indices)
average_similarity

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7385  0.7353  0.7375  0.7375  0.7373  0.7372  0.0011  
Fit time          23.34   21.43   25.47   22.64   29.17   24.41   2.72    
Test time         4.83    3.34    3.79    5.30    3.12    4.08    0.85    
{'test_rmse': array([0.73850347, 0.73528242, 0.7374635 , 0.73753792, 0.73734216]), 'fit_time': (23.33500051498413, 21.43100380897522, 25.471997499465942, 22.635960340499878, 29.169997453689575), 'test_time': (4.833484411239624, 3.33799409866333, 3.7860007286071777, 5.298000812530518, 3.1210033893585205)}


0.41550206306433424

### Adding Location as City and try


In [56]:
def categorize_rating(rating):
    if rating <= 4:
        return 'Low'
    elif rating <= 7:
        return 'Medium'
    elif rating == 0:
        return 'Unknown'
    else:
        return 'High'



import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

users['Location'] = users['Location'].str.lower() ## converitng location into lower coses. 
users.loc[:, 'Location'] = users['Location'].str.replace(' ', '', regex=True)

# Split and clean the 'Location' column in users_df
users[['City', 'State', 'Country']] = users['Location'].str.split(',', n=2, expand=True)
users['City'] = users['City'].str.strip()
users['State'] = users['State'].str.strip()
users['Country'] = users['Country'].str.strip()

# Merge datasets
books_with_ratings = pd.merge(books, ratings, on='ISBN')
books_with_ratings = pd.merge(books_with_ratings, users, on='User-ID')

books_with_ratings['Rating-Category'] = books_with_ratings['Book-Rating'].apply(categorize_rating)
rating_to_numeric = {'Low': 1, 'Medium': 2, 'High': 3, 'Unknown': 0}
books_with_ratings['Numeric-Rating-Category'] = books_with_ratings['Rating-Category'].map(rating_to_numeric)

median_age = books_with_ratings['Age'].median()
books_with_ratings['Age'].fillna(median_age, inplace=True)


books_with_ratings['Year-Of-Publication'] = pd.to_numeric(books_with_ratings['Year-Of-Publication'], errors='coerce')

# Fill missing values
books_with_ratings['Book-Author'].fillna('Unknown', inplace=True)
books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)
books_with_ratings['Location'].fillna('Unknown', inplace=True)


# Convert 'Year-Of-Publication' to numeric, handling non-numeric entries

books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)

# Preprocessing pipeline for additional features
preprocessor = ColumnTransformer(
    transformers=[
        ('author', OneHotEncoder(handle_unknown='ignore'), ['Book-Author']),
        ('city', OneHotEncoder(handle_unknown='ignore'), ['City']),
        ('year', StandardScaler(), ['Year-Of-Publication'])
    ])

# Fit and transform the data
additional_features = preprocessor.fit_transform(books_with_ratings)

# Content-Based Recommender for Book Titles
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])

# Combine title features with additional features
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
combined_features = hstack([count_matrix, additional_features])

combined_features_csr = csr_matrix(combined_features)


# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'Book-Title', 'Numeric-Rating-Category']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Update indices after combining features
indices = pd.Series(range(len(books_with_ratings)), index=books_with_ratings['Book-Title']).drop_duplicates()

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    if book_title not in indices:
        return f"Book title '{book_title}' not found in the dataset."

    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(combined_features_csr[idx], combined_features_csr)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings.iloc[i]['Book-Title']
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings.iloc[i]['ISBN']).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
print(get_recommendations(276726, "Rebecca"))


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['Rebecca', 'The Key to Rebecca', 'Rebecca (Compact Books)', "Rebecca (The world's best reading)", 'My Cousin Rachel (Common Reader Editions)', 'Q', 'Cry No More', 'Under Fire', 'Now You See It . . .', 'Twelve']


In [51]:

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

svd = SVD()

# Evaluate the model using cross-validation
cross_val_results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)
print(cross_val_results)



import numpy as np

def get_feature_vector(book_title, feature_matrix, book_indices):
    # Retrieve the row from the feature matrix corresponding to the given book title
    idx = book_indices[book_title]
    return feature_matrix[idx]

def calculate_average_similarity(input_book_vector, recommended_books, feature_matrix, book_indices):
    similarities = []
    for book in recommended_books:
        book_vector = get_feature_vector(book, feature_matrix, book_indices)
        if book_vector is not None:
            similarity = cosine_similarity(input_book_vector, book_vector)
            similarities.append(similarity[0][0])
    return np.mean(similarities) if similarities else 0

# Example usage
input_book_vector = get_feature_vector("Classical Mythology", combined_features_csr, indices)
recommended_books = get_recommendations(276726, "Classical Mythology")
average_similarity = calculate_average_similarity(input_book_vector, recommended_books, combined_features_csr, indices)
average_similarity

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7365  0.7357  0.7382  0.7379  0.7358  0.7368  0.0011  
Fit time          25.58   26.12   28.27   28.07   27.18   27.04   1.06    
Test time         6.44    3.32    7.65    10.13   3.49    6.20    2.58    
{'test_rmse': array([0.736534  , 0.73565711, 0.73819757, 0.7379298 , 0.73579525]), 'fit_time': (25.58199906349182, 26.116001844406128, 28.269001960754395, 28.072001457214355, 27.17755365371704), 'test_time': (6.436000347137451, 3.3169987201690674, 7.64788031578064, 10.12947678565979, 3.4851627349853516)}


0.41550206306433424

### Removing zero ratings rows and trying

In [ ]:
# To remove rows in the ratings DataFrame where a specific ISBN has only zero ratings and no non-zero ratings, you can follow these steps:
#To remove records from the ratings DataFrame where there is only a single user associated with a single ISBN and that rating is zero, you'll need to:

def handle_ratings(ratings):
    print(ratings)

    # Group by 'ISBN' and check if any of the ratings are non-zero
    isbn_with_non_zero_ratings = ratings.groupby('ISBN')['Book-Rating'].any()
    print(isbn_with_non_zero_ratings)
    # Filter out ISBNs that have only zero ratings
    ratings = ratings[ratings['ISBN'].isin(isbn_with_non_zero_ratings[isbn_with_non_zero_ratings].index)]
    print(ratings)
    
        # Step 1: Identify users with only one rating
    user_rating_counts = ratings['User-ID'].value_counts()
    single_rating_users = user_rating_counts[user_rating_counts == 1].index

    # Step 2: Filter ratings to include only single rating users
    single_ratings = ratings[ratings['User-ID'].isin(single_rating_users)]

    # Step 3: Identify single ratings that are zero
    zero_rating_single_users = single_ratings[single_ratings['Book-Rating'] == 0]['User-ID']

    # Step 4: Remove these records from the original DataFrame
    ratings = ratings[~((ratings['User-ID'].isin(zero_rating_single_users)) & (ratings['Book-Rating'] == 0))]
    
    print(ratings)
    
    return ratings

In [64]:
def categorize_rating(rating):
    if rating <= 4:
        return 'Low'
    elif rating <= 7:
        return 'Medium'
    #elif rating == 0:
    #    return 'Unknown'
    else:
        return 'High'



import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import train_test_split

# Load data
books = pd.read_csv('Books.csv', encoding='latin-1')
ratings = pd.read_csv('Ratings.csv', encoding='latin-1')
users = pd.read_csv('Users.csv', encoding='latin-1')

users['Location'] = users['Location'].str.lower() ## converitng location into lower coses. 
users.loc[:, 'Location'] = users['Location'].str.replace(' ', '', regex=True)

# Split and clean the 'Location' column in users_df
users[['City', 'State', 'Country']] = users['Location'].str.split(',', n=2, expand=True)
users['City'] = users['City'].str.strip()
users['State'] = users['State'].str.strip()
users['Country'] = users['Country'].str.strip()


#handling ratings 
ratings = handle_ratings(ratings)



# Merge datasets
books_with_ratings = pd.merge(books, ratings, on='ISBN')
books_with_ratings = pd.merge(books_with_ratings, users, on='User-ID')

books_with_ratings['Rating-Category'] = books_with_ratings['Book-Rating'].apply(categorize_rating)
#rating_to_numeric = {'Low': 1, 'Medium': 2, 'High': 3, 'Unknown': 0}
rating_to_numeric = {'Low': 1, 'Medium': 2, 'High': 3}
books_with_ratings['Numeric-Rating-Category'] = books_with_ratings['Rating-Category'].map(rating_to_numeric)

median_age = books_with_ratings['Age'].median()
books_with_ratings['Age'].fillna(median_age, inplace=True)


books_with_ratings['Year-Of-Publication'] = pd.to_numeric(books_with_ratings['Year-Of-Publication'], errors='coerce')

# Fill missing values
books_with_ratings['Book-Author'].fillna('Unknown', inplace=True)
books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)
books_with_ratings['Location'].fillna('Unknown', inplace=True)


# Convert 'Year-Of-Publication' to numeric, handling non-numeric entries

books_with_ratings['Year-Of-Publication'].fillna(books_with_ratings['Year-Of-Publication'].median(), inplace=True)

# Preprocessing pipeline for additional features
preprocessor = ColumnTransformer(
    transformers=[
        ('author', OneHotEncoder(handle_unknown='ignore'), ['Book-Author']),
        ('city', OneHotEncoder(handle_unknown='ignore'), ['City']),
        ('year', StandardScaler(), ['Year-Of-Publication'])
    ])

# Fit and transform the data
additional_features = preprocessor.fit_transform(books_with_ratings)

# Content-Based Recommender for Book Titles
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(books_with_ratings['Book-Title'])

# Combine title features with additional features
from scipy.sparse import hstack
from scipy.sparse import csr_matrix
combined_features = hstack([count_matrix, additional_features])

combined_features_csr = csr_matrix(combined_features)


# Collaborative Filtering
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(books_with_ratings[['User-ID', 'Book-Title', 'Numeric-Rating-Category']], reader)
trainset, testset = train_test_split(data, test_size=0.25)
svd = SVD()
svd.fit(trainset)

# Update indices after combining features
indices = pd.Series(range(len(books_with_ratings)), index=books_with_ratings['Book-Title']).drop_duplicates()

# Hybrid Recommendation Function
def get_recommendations(user_id, book_title):
    if book_title not in indices:
        return f"Book title '{book_title}' not found in the dataset."

    idx = indices[book_title]

    # Calculate cosine similarity for the selected book
    cosine_sim = cosine_similarity(combined_features_csr[idx], combined_features_csr)

    sim_scores = list(enumerate(cosine_sim[0]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:100]

    # Predict ratings and filter unique books
    books_predictions = {}
    for i, _ in sim_scores:
        title = books_with_ratings.iloc[i]['Book-Title']
        if title not in books_predictions:
            books_predictions[title] = svd.predict(user_id, books_with_ratings.iloc[i]['ISBN']).est

    # Sort books by predicted rating and select top 10
    top_books = sorted(books_predictions, key=books_predictions.get, reverse=True)[:10]

    return top_books

# Example usage
print(get_recommendations(276726, "Rebecca"))


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


         User-ID         ISBN  Book-Rating
0         276725   034545104X            0
1         276726   0155061224            5
2         276727   0446520802            0
3         276729   052165615X            3
4         276729   0521795028            6
...          ...          ...          ...
1149775   276704   1563526298            9
1149776   276706   0679447156            0
1149777   276709   0515107662           10
1149778   276721   0590442449           10
1149779   276723  05162443314            8

[1149780 rows x 3 columns]
ISBN
 0330299891      True
 0375404120      True
 0586045007     False
 9022906116      True
 9032803328     False
                ...  
cn113107        False
ooo7156103       True
Â§423350229     False
Â´3499128624     True
ÃÂ½crosoft      True
Name: Book-Rating, Length: 340556, dtype: bool
         User-ID         ISBN  Book-Rating
0         276725   034545104X            0
1         276726   0155061224            5
2         276727   0446520802    

In [65]:

from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

svd = SVD()

# Evaluate the model using cross-validation
cross_val_results = cross_validate(svd, data, measures=['RMSE'], cv=5, verbose=True)
print(cross_val_results)



import numpy as np

def get_feature_vector(book_title, feature_matrix, book_indices):
    # Retrieve the row from the feature matrix corresponding to the given book title
    idx = book_indices[book_title]
    return feature_matrix[idx]

def calculate_average_similarity(input_book_vector, recommended_books, feature_matrix, book_indices):
    similarities = []
    for book in recommended_books:
        book_vector = get_feature_vector(book, feature_matrix, book_indices)
        if book_vector is not None:
            similarity = cosine_similarity(input_book_vector, book_vector)
            similarities.append(similarity[0][0])
    return np.mean(similarities) if similarities else 0

# Example usage
input_book_vector = get_feature_vector("Classical Mythology", combined_features_csr, indices)
recommended_books = get_recommendations(276726, "Classical Mythology")
average_similarity = calculate_average_similarity(input_book_vector, recommended_books, combined_features_csr, indices)
average_similarity

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.7745  0.7750  0.7731  0.7757  0.7744  0.7745  0.0008  
Fit time          14.08   14.05   14.16   17.49   13.87   14.73   1.39    
Test time         3.20    3.21    1.84    2.74    2.48    2.70    0.51    
{'test_rmse': array([0.77451942, 0.7750083 , 0.77309069, 0.77565715, 0.77438842]), 'fit_time': (14.076028823852539, 14.049004077911377, 14.157811880111694, 17.491998195648193, 13.865998983383179), 'test_time': (3.2026917934417725, 3.2109971046447754, 1.8419890403747559, 2.73699951171875, 2.4829983711242676)}


0.31050545169770155

 With 0 rating as Unknown , We got similarity as .31, without  rating as unknow , we got similarity as same . SO 

In [66]:
## HYer Arameter Tuning 

from surprise.model_selection import GridSearchCV

param_grid = {
    'n_epochs': [5, 10, 20],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.4, 0.6]
}
gs = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

# Use the best parameters for SVD
algo = SVD(n_epochs=gs.best_params['rmse']['n_epochs'], lr_all=gs.best_params['rmse']['lr_all'], reg_all=gs.best_params['rmse']['reg_all'])


0.7752384163617441
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4}
